In [ ]:
def process_parquet_files_Port_2(parquet_files, export_path, speriod, samples, rps_values,Cat):
    partial_folder_path = os.path.join(processing_folder_path, 'partial')
    concatenated_folder_path = os.path.join(processing_folder_path, 'concatenated')

    os.makedirs(partial_folder_path, exist_ok=True)
    os.makedirs(concatenated_folder_path, exist_ok=True)

    # Initialize an empty list to store the results
    final_grouped_table_1 = []

    # Process each Parquet file individually
    for file in parquet_files:
        # Read the Parquet file into a PyArrow Table
        table = pq.read_table(file)

        grouped_table_1 = table.group_by(['EventId', 'PeriodId', 'EventDate','Admin1Name','Admin1Id']).aggregate([('Loss', 'sum')])
        grouped_table_1 = grouped_table_1.rename_columns(['EventId', 'PeriodId', 'EventDate','Admin1Name','Admin1Id','Sum_Loss'])
    
        # Write intermediate results to disk
        pq.write_table(grouped_table_1, os.path.join(partial_folder_path, f'grouped_table_1_{os.path.basename(file)}'))

    # Read all intermediate files and concatenate them
    intermediate_files_1 = [os.path.join(partial_folder_path, f) for f in os.listdir(partial_folder_path) if f.startswith('grouped_table_1_')]

    final_grouped_table_1 = [pq.read_table(f) for f in intermediate_files_1]

    final_table_1 = pa.concat_tables(final_grouped_table_1)

    # Perform final grouping and sorting
    f_grouped_table_1 = final_table_1.group_by(['EventId', 'PeriodId', 'EventDate', 'Admin1Name', 'Admin1Id']).aggregate([('Sum_Loss', 'sum')])
    sorted_final_table_1 = f_grouped_table_1.sort_by([('Sum_Loss_sum', 'descending')])

    # Get distinct Admin1Id and Admin1Name
    distinct_admins = sorted_final_table_1.select(['Admin1Id', 'Admin1Name']).to_pandas().drop_duplicates()

    for idx, row in distinct_admins.iterrows():
        admin1_id = row['Admin1Id']
        admin1_name = row['Admin1Name']

        # Filter the table for the current Admin1Id and Admin1Name
        filtered_table = sorted_final_table_1.filter(pa.compute.equal(sorted_final_table_1['Admin1Id'], admin1_id))

        # Convert to pandas DataFrame
        dataframe_1 = filtered_table.to_pandas()
        calculations_on_dataframe_1(dataframe_1)  # Perform additional calculations
        final_df_EP_Portfolio_GU = pd.concat([final_df_2, final_df_3], ignore_index=True)
        new_ep_type_order = ["OEP", "AEP", "TCE-OEP", "TCE-AEP"]
        final_df_EP_Portfolio_GU['EPType'] = pd.Categorical(final_df_EP_Portfolio_GU['EPType'], categories=new_ep_type_order, ordered=True)
        final_df_EP_Portfolio_GU = final_df_EP_Portfolio_GU.sort_values(by=['EPType', 'ReturnPeriod'], ascending=[True, False]).reset_index(drop=True)
        final_df_EP_Portfolio_GU['Admin1Id'] = final_df_EP_Portfolio_GU['Admin1Id'].astype('int64')
        final_df_EP_Portfolio_GU['Admin1Id'] = final_df_EP_Portfolio_GU['Admin1Id'].apply(lambda x: Decimal(x))

        # Define the schema to match the required Parquet file schema
        schema = pa.schema([
            pa.field('EPType', pa.string(), nullable=True),
            pa.field('Loss', pa.float64(), nullable=True),
            pa.field('ReturnPeriod', pa.float64(), nullable=True),
            pa.field('Admin1Id', pa.decimal128(38, 0), nullable=True),
            pa.field('Admin1Name', pa.string(), nullable=True),
        ])

        # Convert DataFrame to Arrow Table with the specified schema
        table = pa.Table.from_pandas(final_df_EP_Portfolio_GU, schema=schema)
        # Save to Parquet
        #pq.write_table(table, parquet_file_path)
        #print(f"Parquet file saved successfully at {parquet_file_path}")


        #FOR GU

        export_path =os.path.join(main_folder_path,'EP','Admin1',Cat)
        parquet_file_path = os.path.join(export_path,f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Admin1_{Cat}_{idx}.parquet')
        pq.write_table(table, parquet_file_path)
try:
    process_parquet_files_Port_2(parquet_files_grp, export_path, speriod, samples, rps_values,"GU")
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing : {e}")
    pass

#FOR GR
try:
    process_parquet_files_Port_2(parquet_files_grp_gr, export_path_GR, speriod, samples, rps_values,"GR")
except (NameError, AttributeError,ValueError) as e:
    print(f"Error processing : {e}")
    pass

